In [1]:
import re
import pandas as pd
import numpy as np
import string

import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\renat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Base de dados de treino

In [2]:
df = pd.read_csv('/Users/renat/Downloads/imdb-reviews-pt-br.csv')
df.shape

(49459, 4)

In [3]:
df.columns = ['id', 'text_en', 'Text', 'Classificacao']
dic_setment = {
    'neg' : 0,
    'pos': 1
}
df = df.replace(dic_setment)
df['Classificacao'].value_counts()

0    24765
1    24694
Name: Classificacao, dtype: int64

In [3]:
# remoção de features inrelevantes
df = df.drop(columns=['Unnamed: 0', 'Created At', 'Geo Coordinates.latitude',
       'Geo Coordinates.longitude', 'User Location', 'Username',
       'User Screen Name', 'Retweet Count', 'Observação',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'])

## Feature Engineering

### Features que serão adicionadas
    1 - Número de palavras no texto
    2 - Número de palavras únicas no texto
    3 - Número de caracteres no texto
    4 - Número de palavras irrelevantes (stopwords)
    5 - Número de pontuações
    6 - Número palavras maiúsculas
    7 - Número de palavras de caixa de título
    8 - Comprimento médio das palavras

In [47]:
# Numero de palavras no texto
df["num_words"] = df['Text'].apply(lambda x: len(str(x).split()))

# Numero de palavras unicas no texto
df['num_unique_words'] = df['Text'].apply(lambda x: len(set(str(x).split())))

# Número de caracteres no texto
df['num_chars'] = df['Text'].apply(lambda x: len(str(x)))

# Número de palavras irrelevantes (stopwords)
df['num_stopwords'] = df['Text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords]))

# Número de pontuações
df['num_punctuations'] = df['Text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# Número palavras maiúsculas
df['num_words_upper'] = df['Text'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

# Número de palavras de caixa de título
df['num_words_title'] = df['Text'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

# Comprimento médio das palavras
df['mean_word_len'] = df['Text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))


## Data Preprocessing
- Converter texto para minúsculo
- Remover pontuações
- Limpeza de Número 
- remoção de stopwords

In [5]:
# Removing punctuations
punctuation_list =[',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', 
        '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 
        '█', '…', '“', '★', '”', '–', '●', '►', '−', '¢', '¬', '░', '¡', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
        '—', '‹', '─', '▒', '：', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '⋅', '‘', '∞', 
        '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '・', '╦', '╣', '╔', '╗', '▬', '❤', '≤', '‡', '√', '◄', '━', 
        '⇒', '▶', '≥', '╝', '♡', '◊', '。', '✈', '≡', '☺', '✔', '↵', '≈', '✓', '♣', '☎', '℃', '◦', '└', '‟', '～', '！', '○', 
        '◆', '№', '♠', '▌', '✿', '▸', '⁄', '□', '❖', '✦', '．', '÷', '｜', '┃', '／', '￥', '╠', '↩', '✭', '▐', '☼', '☻', '┐', 
        '├', '«', '∼', '┌', '℉', '☮', '฿', '≦', '♬', '✧', '〉', '－', '⌂', '✖', '･', '◕', '※', '‖', '◀', '‰', '\x97', '↺', 
        '∆', '┘', '┬', '╬', '،', '⌘', '⊂', '＞', '〈', '⎙', '？', '☠', '⇐', '▫', '∗', '∈', '≠', '♀', '♔', '˚', '℗', '┗', '＊', 
        '┼', '❀', '＆', '∩', '♂', '‿', '∑', '‣', '➜', '┛', '⇓', '☯', '⊖', '☀', '┳', '；', '∇', '⇑', '✰', '◇', '♯', '☞', '´', 
        '↔', '┏', '｡', '◘', '∂', '✌', '♭', '┣', '┴', '┓', '✨', '\xa0', '˜', '❥', '┫', '℠', '✒', '［', '∫', '\x93', '≧', '］', 
        '\x94', '∀', '♛', '\x96', '∨', '◎', '↻', '⇩', '＜', '≫', '✩', '✪', '♕', '؟', '₤', '☛', '╮', '␊', '＋', '┈', '％', 
        '╋', '▽', '⇨', '┻', '⊗', '￡', '।', '▂', '✯', '▇', '＿', '➤', '✞', '＝', '▷', '△', '◙', '▅', '✝', '∧', '␉', '☭', 
        '┊', '╯', '☾', '➔', '∴', '\x92', '▃', '↳', '＾', '׳', '➢', '╭', '➡', '＠', '⊙', '☢', '˝', '∏', '„', '∥', '❝', '☐', 
        '▆', '╱', '⋙', '๏', '☁', '⇔', '▔', '\x91', '➚', '◡', '╰', '\x85', '♢', '˙', '۞', '✘', '✮', '☑', '⋆', 'ⓘ', '❒', 
        '☣', '✉', '⌊', '➠', '∣', '❑', '◢', 'ⓒ', '\x80', '〒', '∕', '▮', '⦿', '✫', '✚', '⋯', '♩', '☂', '❞', '‗', '܂', '☜', 
        '‾', '✜', '╲', '∘', '⟩', '＼', '⟨', '·', '✗', '♚', '∅', 'ⓔ', '◣', '͡', '‛', '❦', '◠', '✄', '❄', '∃', '␣', '≪', '｢', 
        '≅', '◯', '☽', '∎', '｣', '❧', '̅', 'ⓐ', '↘', '⚓', '▣', '˘', '∪', '⇢', '✍', '⊥', '＃', '⎯', '↠', '۩', '☰', '◥', 
        '⊆', '✽', '⚡', '↪', '❁', '☹', '◼', '☃', '◤', '❏', 'ⓢ', '⊱', '➝', '̣', '✡', '∠', '｀', '▴', '┤', '∝', '♏', 'ⓐ', 
        '✎', ';', '␤', '＇', '❣', '✂', '✤', 'ⓞ', '☪', '✴', '⌒', '˛', '♒', '＄', '✶', '▻', 'ⓔ', '◌', '◈', '❚', '❂', '￦', 
        '◉', '╜', '̃', '✱', '╖', '❉', 'ⓡ', '↗', 'ⓣ', '♻', '➽', '׀', '✲', '✬', '☉', '▉', '≒', '☥', '⌐', '♨', '✕', 'ⓝ', 
        '⊰', '❘', '＂', '⇧', '̵', '➪', '▁', '▏', '⊃', 'ⓛ', '‚', '♰', '́', '✏', '⏑', '̶', 'ⓢ', '⩾', '￠', '❍', '≃', '⋰', '♋', 
        '､', '̂', '❋', '✳', 'ⓤ', '╤', '▕', '⌣', '✸', '℮', '⁺', '▨', '╨', 'ⓥ', '♈', '❃', '☝', '✻', '⊇', '≻', '♘', '♞', 
        '◂', '✟', '⌠', '✠', '☚', '✥', '❊', 'ⓒ', '⌈', '❅', 'ⓡ', '♧', 'ⓞ', '▭', '❱', 'ⓣ', '∟', '☕', '♺', '∵', '⍝', 'ⓑ', 
        '✵', '✣', '٭', '♆', 'ⓘ', '∶', '⚜', '◞', '்', '✹', '➥', '↕', '̳', '∷', '✋', '➧', '∋', '̿', 'ͧ', '┅', '⥤', '⬆', '⋱', 
        '☄', '↖', '⋮', '۔', '♌', 'ⓛ', '╕', '♓', '❯', '♍', '▋', '✺', '⭐', '✾', '♊', '➣', '▿', 'ⓑ', '♉', '⏠', '◾', '▹', 
        '⩽', '↦', '╥', '⍵', '⌋', '։', '➨', '∮', '⇥', 'ⓗ', 'ⓓ', '⁻', '⎝', '⌥', '⌉', '◔', '◑', '✼', '♎', '♐', '╪', '⊚', 
        '☒', '⇤', 'ⓜ', '⎠', '◐', '⚠', '╞', '◗', '⎕', 'ⓨ', '☟', 'ⓟ', '♟', '❈', '↬', 'ⓓ', '◻', '♮', '❙', '♤', '∉', '؛', 
        '⁂', 'ⓝ', '־', '♑', '╫', '╓', '╳', '⬅', '☔', '☸', '┄', '╧', '׃', '⎢', '❆', '⋄', '⚫', '̏', '☏', '➞', '͂', '␙', 
        'ⓤ', '◟', '̊', '⚐', '✙', '↙', '̾', '℘', '✷', '⍺', '❌', '⊢', '▵', '✅', 'ⓖ', '☨', '▰', '╡', 'ⓜ', '☤', '∽', '╘', 
        '˹', '↨', '♙', '⬇', '♱', '⌡', '⠀', '╛', '❕', '┉', 'ⓟ', '̀', '♖', 'ⓚ', '┆', '⎜', '◜', '⚾', '⤴', '✇', '╟', '⎛', 
        '☩', '➲', '➟', 'ⓥ', 'ⓗ', '⏝', '◃', '╢', '↯', '✆', '˃', '⍴', '❇', '⚽', '╒', '̸', '♜', '☓', '➳', '⇄', '☬', '⚑', 
        '✐', '⌃', '◅', '▢', '❐', '∊', '☈', '॥', '⎮', '▩', 'ு', '⊹', '‵', '␔', '☊', '➸', '̌', '☿', '⇉', '⊳', '╙', 'ⓦ', 
        '⇣', '｛', '̄', '↝', '⎟', '▍', '❗', '״', '΄', '▞', '◁', '⛄', '⇝', '⎪', '♁', '⇠', '☇', '✊', 'ி', '｝', '⭕', '➘', 
        '⁀', '☙', '❛', '❓', '⟲', '⇀', '≲', 'ⓕ', '⎥', '\u06dd', 'ͤ', '₋', '̱', '̎', '♝', '≳', '▙', '➭', '܀', 'ⓖ', '⇛', '▊', 
        '⇗', '̷', '⇱', '℅', 'ⓧ', '⚛', '̐', '̕', '⇌', '␀', '≌', 'ⓦ', '⊤', '̓', '☦', 'ⓕ', '▜', '➙', 'ⓨ', '⌨', '◮', '☷', 
        '◍', 'ⓚ', '≔', '⏩', '⍳', '℞', '┋', '˻', '▚', '≺', 'ْ', '▟', '➻', '̪', '⏪', '̉', '⎞', '┇', '⍟', '⇪', '▎', '⇦', '␝', 
        '⤷', '≖', '⟶', '♗', '̴', '♄', 'ͨ', '̈', '❜', '̡', '▛', '✁', '➩', 'ா', '˂', '↥', '⏎', '⎷', '̲', '➖', '↲', '⩵', '̗', '❢', 
        '≎', '⚔', '⇇', '̑', '⊿', '̖', '☍', '➹', '⥊', '⁁', '✢']

In [6]:
def remove_punctuation(text):
    for punctuation in punctuation_list:
        if punctuation in text:
            text = text.replace(punctuation, f'{punctuation}')
    return text

In [7]:
def clean_numbers(text):
    if bool(re.search(r'\d', text)):
        text = re.sub('[0-9]{5,}', '#####', text)
        text = re.sub('[0-9]{4}', '####', text)
        text = re.sub('[0-9]{3}', '###', text)
        text = re.sub('[0-9]{2}', '##', text)
    return text

In [8]:
from nltk.tokenize.toktok import ToktokTokenizer

def remove_stopwords(text, is_lower_case=True):
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [9]:
# Aplica todos os preprocessamentos 
def clean_text(x):
    x = x.lower()
    x = remove_punctuation(x)
    x = clean_numbers(x)
    x = remove_stopwords(x)
    x = x.replace("'", "")
    return x

In [48]:
df['preprocessed_text'] = df['Text'].apply(lambda x: clean_text(x))


## Count Vectorize e Modelo de Regreção Logistica

In [11]:
import copy
from sklearn.metrics._classification import log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn import metrics

In [12]:
# diferenciano textos com polaridade negativa dos demais
dic_classificacao = {
    'Neutro' : 0,
    'Negativo': 2,
    'Positivo': 1,
}
df['Classificacao'] = df['Classificacao'].replace(dic_classificacao)

In [49]:
train, test = np.split(df.sample(frac=1), [int(.8*len(df))])
train.shape, test.shape

((39567, 13), (9892, 13))

In [50]:
# Creating CountVectorizer object
vectorizer = CountVectorizer(
    dtype=np.float32, 
    strip_accents='unicode', 
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 3),
    min_df=3
)
# Fit the vectorizer on training data after preprocessing
vectorizer.fit_transform(train['preprocessed_text'].values.tolist() + test['preprocessed_text'].values.tolist())
train_vectorizer = vectorizer.transform(train['preprocessed_text'].values.tolist())
test_vectorizer = vectorizer.transform(test['preprocessed_text'].values.tolist())

In [51]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
kf.split(train)

<generator object _BaseKFold.split at 0x000002B02DD9DA50>

In [52]:
train_y = train['Classificacao'].values

In [53]:
modelo = LogisticRegression(C=5., solver='sag')
modelo.fit(train_vectorizer, train_y)

C:\Users\renat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=5.0, solver='sag')

In [18]:
resultado = modelo.predict(test_vectorizer)

In [55]:
# Medino a acuracia do modelo
from sklearn.metrics import accuracy_score

accuracy_score(test['Classificacao'].values, resultado)

ValueError: Found input variables with inconsistent numbers of samples: [9892, 1640]

## Salvando Modelo

In [28]:

# salvar modelo
import pickle
 
# salvar o modelo 
with open('../data/LogisticRegression_AS.pkl', 'wb') as file:
    pickle.dump(modelo, file)

# salvar o vectorizer
with open('../data/text_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)